In [ ]:
!python -V

Python 3.9.18


In [ ]:
import pandas as pd

In [ ]:
import pickle

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_parquet('C:\\Users\\Lotos\\Downloads\\yellow_tripdata_2023-01.parquet')
print(df.head(3))

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [51]:
num_columns = len(df.columns)
print("Number of columns:", num_columns)

Number of columns: 19


In [52]:
# Drop outliers
df_original = df.copy()
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records left
fraction_remaining = df.shape[0] / len(df_original)
print(f"Fraction of records left: {fraction_remaining}")



Fraction of records left: 0.9812202822125979


In [53]:
from sklearn.feature_extraction import DictVectorizer

# Convert dataframe to a list of dictionaries
dict_list = df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Fit a dictionary vectorizer with sparse output
vectorizer = DictVectorizer(sparse=True)
feature_matrix = vectorizer.fit_transform(dict_list)

# Get the dimensionality of the feature matrix
num_columns = feature_matrix.shape[1]

print("Dimensionality of the feature matrix:", num_columns)


Dimensionality of the feature matrix: 515


In [56]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Train-test split
X_train = feature_matrix
y_train = df['duration']

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred = model.predict(X_train)

# Calculate RMSE on training data
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

print("RMSE on train:", rmse_train)


 

RMSE on train: 7.649262060255514


In [59]:
df_val = pd.read_parquet('C:\\Users\\Lotos\\Downloads\\yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]
# Convert validation dataframe to a list of dictionaries
dict_list_val = df_val[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Transform validation data using the same vectorizer
X_val = vectorizer.transform(dict_list_val)

# Make predictions on the validation data
y_val_pred = model.predict(X_val)

# Calculate RMSE on validation data
y_val = df_val['duration']
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))

print("RMSE on validation:", rmse_val)


RMSE on validation: 7.811816183354732
